In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd 
import numpy as np 
import pickle 

# Define Model

In [2]:
class lstm_Classifier(nn.Module):
    def __init__ (self, input_size, hidden_size, num_layers, batch_size, dropout, num_classes):
        super(lstm_Classifier, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.dropout = dropout
        self.num_classes = num_classes

        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, dropout= dropout)
        self.fc = nn.Linear(hidden_size,num_classes)

    def init_hidden(self):
        hidden = (torch.randn(self.num_layers, self.batch_size, self.hidden_size),
                torch.randn(self.num_layers, self.batch_size, self.hidden_size))
        return hidden

    def forward(self, X, hidden = None): 
        
        if hidden == None: 
            hidden = self.init_hidden()

        X,_ = self.lstm(X, hidden)
        X = self.fc(X[-1])

        return X

In [3]:
input_size = 4
num_layers = 2
hidden_size = 4
batch_size = 6
lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, dropout= .05) # input = [Open, HIgh, low, Close] output = [Up, Neutral, Down]

In [4]:
model = lstm_Classifier(input_size = input_size, hidden_size = hidden_size,
                                 num_layers = num_layers,batch_size=batch_size, dropout= .05, num_classes=3)

In [8]:

sequence_len = 2
inputs = [torch.randn(sequence_len, batch_size, input_size) for _ in range(1)]

# initialize the hidden state.
hidden = (torch.randn(num_layers, batch_size, hidden_size),
          torch.randn(num_layers, batch_size, hidden_size)) 

for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    
    # out, hidden = lstm(i.view(sequence_len, batch_size, input_size), hidden)
    # out, hidden = lstm(i, hidden)
    out, hidden = lstm(i)
    print(out)
    
    out = model(i)

    print(out)
    # print(hidden)

tensor([[[-0.0940, -0.0892,  0.0322, -0.1542],
         [-0.0925, -0.0890,  0.0293, -0.1356],
         [-0.0927, -0.0819,  0.0286, -0.1473],
         [-0.0858, -0.0929,  0.0245, -0.1288],
         [-0.0964, -0.0844,  0.0318, -0.1473],
         [-0.0989, -0.0907,  0.0423, -0.1400]],

        [[-0.1274, -0.1215,  0.0629, -0.2094],
         [-0.1267, -0.1213,  0.0553, -0.1822],
         [-0.1324, -0.1176,  0.0624, -0.1940],
         [-0.1284, -0.1213,  0.0552, -0.1715],
         [-0.1346, -0.1296,  0.0620, -0.2115],
         [-0.1336, -0.1166,  0.0721, -0.1923]]], grad_fn=<StackBackward>)
tensor([[[-0.4430,  0.3265,  0.1664],
         [-0.4156,  0.2622,  0.2493],
         [-0.4649,  0.3541,  0.1378],
         [-0.5444,  0.3608,  0.1031],
         [-0.4267,  0.4208,  0.1373],
         [-0.3404,  0.4936,  0.1315]],

        [[-0.5383,  0.3504,  0.0992],
         [-0.4056,  0.3152,  0.2181],
         [-0.5047,  0.3502,  0.1228],
         [-0.5228,  0.4002,  0.0870],
         [-0.4769,  0.379

In [12]:
print(out[-1])

tensor([[-0.5383,  0.3504,  0.0992],
        [-0.4056,  0.3152,  0.2181],
        [-0.5047,  0.3502,  0.1228],
        [-0.5228,  0.4002,  0.0870],
        [-0.4769,  0.3791,  0.1269],
        [-0.4238,  0.4240,  0.1264]], grad_fn=<SelectBackward>)
